In [3]:
import pandas as pd
import yfinance as yf
import numpy as np

In [4]:
data = pd.read_excel("Bist100_duznelenmis.xlsx",header=0)
data.head()

,Kodlar
0,ADEL.IS
1,ADGYO.IS
2,AGHOL.IS
3,AGROT.IS
4,AHGAZ.IS


In [7]:
tickers = data["Kodlar"].tolist()

In [9]:
stats = ["NetIncomeFromContinuingOperations",
         "TotalAssets",
         "OperatingCashFlow",
         "LongTermDebtAndCapitalLeaseObligation",
         "TotalNonCurrentLiabilitiesNetMinorityInterest",
         "CurrentAssets",
         "CurrentLiabilities",
         "StockholdersEquity",
         "TotalRevenue",
         "GrossProfit"] # change as required

indx = ["NetIncome","TotAssets","CashFlowOps","LTDebt","TotLTLiab",
        "CurrAssets","CurrLiab","CommStock","TotRevenue","GrossProfit"]

In [11]:
financial_dir={}

for ticker in tickers:
    try:
        df1 = yf.Ticker(ticker).get_income_stmt()
        df2 = yf.Ticker(ticker).get_balance_sheet()
        df3 = yf.Ticker(ticker).get_cashflow()
        
        if df1.empty or df2.empty or df3.empty or len(df1.columns)<3:
            print(f"{ticker}: Finansal veri çekilemedi, atlanıyor.")
            continue
        df1 = df1.iloc[:,:3]
        df2 = df2.iloc[:,:3]
        df3 = df3.iloc[:,:3]
        df = pd.concat([df1,df2,df3])
        financial_dir[ticker] = df
        print("data extracted for ",ticker)
        financial_dir[ticker] = df
    except Exception as e:
        print(ticker,":", e)

data extracted for  ADEL.IS
data extracted for  ADGYO.IS
data extracted for  AGHOL.IS
data extracted for  AGROT.IS
data extracted for  AHGAZ.IS
data extracted for  AKCNS.IS
data extracted for  AKFGY.IS
data extracted for  AKFIS.IS
data extracted for  AKFYE.IS
data extracted for  AKSGY.IS
data extracted for  AKSA.IS
data extracted for  AKSEN.IS
data extracted for  ALGYO.IS
data extracted for  ALARK.IS
data extracted for  ALFAS.IS
data extracted for  ALTNY.IS
data extracted for  AEFES.IS
data extracted for  ARCLK.IS
data extracted for  ARDYZ.IS
data extracted for  ARMGD.IS
data extracted for  ASGYO.IS
data extracted for  ASELS.IS
data extracted for  ASTOR.IS
data extracted for  ATAKP.IS
data extracted for  ATATP.IS
data extracted for  AVPGY.IS
data extracted for  AYDEM.IS
data extracted for  AYGAZ.IS
data extracted for  AZTEK.IS
BASGZ.IS: Finansal veri çekilemedi, atlanıyor.
data extracted for  BTCIM.IS
data extracted for  BSOKE.IS
data extracted for  BERA.IS
data extracted for  BJKAS.IS

In [27]:
financial_dir["TUREX.IS"].loc[stats,:].isna().sum().sum()>0

True

In [29]:
def info_filter(df, stats, indx, lookback):
    """Function to filter relevant financial information."""
    missing_stats = [stat for stat in stats if stat not in df.index]
    
    if missing_stats:
        print(f"Skipping due to missing stats: {missing_stats}")
        return pd.DataFrame()  # Boş bir DataFrame döndür

    existing_stats = [stat for stat in stats if stat in df.index]
    df_new = df.loc[existing_stats, df.columns[:lookback]]
    df_new.rename(dict(zip(existing_stats, indx[:len(existing_stats)])), inplace=True)
    df_new.loc["OtherLTDebt", :] = df_new.loc["TotLTLiab", :] - df_new.loc["LTDebt", :]
    
    return df_new

In [31]:
#applying filtering to the finacials
transformed_df = {}
for ticker in financial_dir:
    df_filtered = info_filter(financial_dir[ticker], stats, indx, 3)
    if df_filtered.isna().sum().sum()>0:
        continue
    elif not df_filtered.empty:  # Eğer DataFrame boş değilse ekle
        transformed_df[ticker] = df_filtered

Skipping due to missing stats: ['LongTermDebtAndCapitalLeaseObligation']
Skipping due to missing stats: ['NetIncomeFromContinuingOperations']
Skipping due to missing stats: ['NetIncomeFromContinuingOperations']
Skipping due to missing stats: ['LongTermDebtAndCapitalLeaseObligation', 'GrossProfit']
Skipping due to missing stats: ['NetIncomeFromContinuingOperations', 'LongTermDebtAndCapitalLeaseObligation']
Skipping due to missing stats: ['NetIncomeFromContinuingOperations']
Skipping due to missing stats: ['TotalNonCurrentLiabilitiesNetMinorityInterest', 'CurrentAssets', 'CurrentLiabilities', 'GrossProfit']
Skipping due to missing stats: ['NetIncomeFromContinuingOperations', 'TotalNonCurrentLiabilitiesNetMinorityInterest', 'CurrentAssets', 'CurrentLiabilities', 'GrossProfit']
Skipping due to missing stats: ['NetIncomeFromContinuingOperations', 'TotalNonCurrentLiabilitiesNetMinorityInterest', 'CurrentAssets', 'CurrentLiabilities', 'GrossProfit']
Skipping due to missing stats: ['NetIncomeF

In [33]:
def safe_divide(numerator, denominator, default_value=0):
    """ Bölme işlemi sırasında sıfıra bölünmeyi önlemek için güvenli bölme fonksiyonu. """
    return numerator / denominator if denominator != 0 else default_value

def piotroski_f(df_dict):
    """Function to calculate F-score of each stock and output information as dataframe."""
    f_score = {}
    for ticker, df in df_dict.items():
        if df.empty:  # Eğer DataFrame boşsa atla
            print(f"Skipping {ticker} due to empty dataframe")
            continue
        
        columns = df.columns
        
        # Finansal verilerin eksik olup olmadığını kontrol et
        try:
            tot_assets_0 = df.loc["TotAssets", columns[0]]
            tot_assets_1 = df.loc["TotAssets", columns[1]]
            tot_assets_2 = df.loc["TotAssets", columns[2]]
            
            avg_assets_0_1 = safe_divide((tot_assets_0 + tot_assets_1), 2)
            avg_assets_1_2 = safe_divide((tot_assets_1 + tot_assets_2), 2)
            
            roa_0 = safe_divide(df.loc["NetIncome", columns[0]], avg_assets_0_1)
            roa_1 = safe_divide(df.loc["NetIncome", columns[1]], avg_assets_1_2)
            
            ROA_FS = int(roa_0 > 0)
            CFO_FS = int(df.loc["CashFlowOps", columns[0]] > 0)
            ROA_D_FS = int(roa_0 > roa_1)
            CFO_ROA_FS = int(safe_divide(df.loc["CashFlowOps", columns[0]], tot_assets_0) > roa_0)
            
            LTD_0 = df.loc["LTDebt", columns[0]] + df.loc["OtherLTDebt", columns[0]]
            LTD_1 = df.loc["LTDebt", columns[1]] + df.loc["OtherLTDebt", columns[1]]
            LTD_FS = int(LTD_0 < LTD_1)
            
            CR_0 = safe_divide(df.loc["CurrAssets", columns[0]], df.loc["CurrLiab", columns[0]])
            CR_1 = safe_divide(df.loc["CurrAssets", columns[1]], df.loc["CurrLiab", columns[1]])
            CR_FS = int(CR_0 > CR_1)
            
            DILUTION_FS = int(df.loc["CommStock", columns[0]] <= df.loc["CommStock", columns[1]])
            
            GM_0 = safe_divide(df.loc["GrossProfit", columns[0]], df.loc["TotRevenue", columns[0]])
            GM_1 = safe_divide(df.loc["GrossProfit", columns[1]], df.loc["TotRevenue", columns[1]])
            GM_FS = int(GM_0 > GM_1)
            
            ATO_0 = safe_divide(df.loc["TotRevenue", columns[0]], avg_assets_0_1)
            ATO_1 = safe_divide(df.loc["TotRevenue", columns[1]], avg_assets_1_2)
            ATO_FS = int(ATO_0 > ATO_1)
            
            f_score[ticker] = [ROA_FS, CFO_FS, ROA_D_FS, CFO_ROA_FS, LTD_FS, CR_FS, DILUTION_FS, GM_FS, ATO_FS]
        
        except KeyError as e:
            print(f"{ticker}: Eksik veri nedeniyle atlanıyor: {e}")
            continue  # Eğer eksik veri varsa hisseyi atla
        
    f_score_df = pd.DataFrame(f_score, index=["PosROA", "PosCFO", "ROAChange", "Accruals", "Leverage", "Liquidity", "Dilution", "GM", "ATO"])
    return f_score_df

In [35]:
# sorting stocks with highest Piotroski f score to lowest
f_score_df = piotroski_f(transformed_df)
f_score_df=pd.DataFrame(f_score_df.sum().sort_values(ascending=False))
f_score_df.reset_index(inplace=True)
f_score_df.rename(columns={"index":"Kod",0:"f_score"},inplace=True)
f_score_df.head(30)

,Kod,f_score
0,LOGO.IS,9
1,GSRAY.IS,8
2,KARSN.IS,7
3,SARKY.IS,7
4,ECILC.IS,7
5,BFREN.IS,7
6,TKFEN.IS,7
7,ADEL.IS,6
8,GUBRF.IS,6
9,DOAS.IS,6


In [37]:
f_score_df.to_excel("piotroski_f_scores.xlsx", index=False)

In [39]:
f_score_df.tail(10)

,Kod,f_score
158,GESAN.IS,2
159,TMSN.IS,2
160,REEDR.IS,2
161,PARSN.IS,2
162,NATEN.IS,2
163,PATEK.IS,1
164,BINBN.IS,1
165,IEYHO.IS,1
166,SRVGY.IS,1
167,TUKAS.IS,1


In [30]:
"""
def piotroski_f(df_dict):
    f_score = {}
    for ticker, df in df_dict.items():
        if df.empty:  # Eğer DataFrame boşsa atla
            print(f"Skipping {ticker} due to empty dataframe")
            continue
        columns = df_dict[ticker].columns
        ROA_FS = int(df_dict[ticker].loc["NetIncome",columns[0]]/((df_dict[ticker].loc["TotAssets",columns[0]] + df_dict[ticker].loc["TotAssets",columns[1]])/2) > 0)
        CFO_FS = int(df_dict[ticker].loc["CashFlowOps",columns[0]] > 0)
        ROA_D_FS = int((df_dict[ticker].loc["NetIncome",columns[0]]/((df_dict[ticker].loc["TotAssets",columns[0]] + df_dict[ticker].loc["TotAssets",columns[1]])/2)) > (df_dict[ticker].loc["NetIncome",columns[1]]/((df_dict[ticker].loc["TotAssets",columns[1]] + df_dict[ticker].loc["TotAssets",columns[2]])/2)))
        CFO_ROA_FS = int(df_dict[ticker].loc["CashFlowOps",columns[0]]/df_dict[ticker].loc["TotAssets",columns[0]] > df_dict[ticker].loc["NetIncome",columns[0]]/((df_dict[ticker].loc["TotAssets",columns[0]] + df_dict[ticker].loc["TotAssets",columns[1]])/2))
        LTD_FS = int((df_dict[ticker].loc["LTDebt",columns[0]] + df_dict[ticker].loc["OtherLTDebt",columns[0]]) < (df_dict[ticker].loc["LTDebt",columns[1]] + df_dict[ticker].loc["OtherLTDebt",columns[1]]))
        CR_FS = int((df_dict[ticker].loc["CurrAssets",columns[0]] / df_dict[ticker].loc["CurrLiab",columns[0]]) > (df_dict[ticker].loc["CurrAssets",columns[1]] / df_dict[ticker].loc["CurrLiab",columns[1]]))
        DILUTION_FS = int(df_dict[ticker].loc["CommStock",columns[0]] <= df_dict[ticker].loc["CommStock",columns[1]])
        GM_FS = int((df_dict[ticker].loc["GrossProfit",columns[0]]/df_dict[ticker].loc["TotRevenue",columns[0]]) > (df_dict[ticker].loc["GrossProfit",columns[1]]/df_dict[ticker].loc["TotRevenue",columns[1]]))
        ATO_FS = int((df_dict[ticker].loc["TotRevenue",columns[0]]/((df_dict[ticker].loc["TotAssets",columns[0]] + df_dict[ticker].loc["TotAssets",columns[1]])/2)) > (df_dict[ticker].loc["TotRevenue",columns[1]]/((df_dict[ticker].loc["TotAssets",columns[1]] + df_dict[ticker].loc["TotAssets",columns[2]])/2)))
        f_score[ticker] = [ROA_FS,CFO_FS,ROA_D_FS,CFO_ROA_FS,LTD_FS,CR_FS,DILUTION_FS,GM_FS,ATO_FS]
    f_score_df = pd.DataFrame(f_score,index=["PosROA","PosCFO","ROAChange","Accruals","Leverage","Liquidity","Dilution","GM","ATO"])
    return f_score_df
"""

'\ndef piotroski_f(df_dict):\n    f_score = {}\n    for ticker, df in df_dict.items():\n        if df.empty:  # Eğer DataFrame boşsa atla\n            print(f"Skipping {ticker} due to empty dataframe")\n            continue\n        columns = df_dict[ticker].columns\n        ROA_FS = int(df_dict[ticker].loc["NetIncome",columns[0]]/((df_dict[ticker].loc["TotAssets",columns[0]] + df_dict[ticker].loc["TotAssets",columns[1]])/2) > 0)\n        CFO_FS = int(df_dict[ticker].loc["CashFlowOps",columns[0]] > 0)\n        ROA_D_FS = int((df_dict[ticker].loc["NetIncome",columns[0]]/((df_dict[ticker].loc["TotAssets",columns[0]] + df_dict[ticker].loc["TotAssets",columns[1]])/2)) > (df_dict[ticker].loc["NetIncome",columns[1]]/((df_dict[ticker].loc["TotAssets",columns[1]] + df_dict[ticker].loc["TotAssets",columns[2]])/2)))\n        CFO_ROA_FS = int(df_dict[ticker].loc["CashFlowOps",columns[0]]/df_dict[ticker].loc["TotAssets",columns[0]] > df_dict[ticker].loc["NetIncome",columns[0]]/((df_dict[ticker].lo